#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

github ipynb 출력 오류로 셀 출력 없는 버전이 업로드 되었습니다.

셀 출력 있는 버전은 colab에 업로드 되어있습니다.
링크: https://colab.research.google.com/drive/1kDfYmyZQJFmiXpNMDD-br8ikRXSFSwEX?usp=sharing

In [ ]:
# pip installs
!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft trl matplotlib wandb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('make_data2.py','wb').write(src)

#라이브러리 import

In [ ]:
from pathlib import Path
import os
import json
import pickle
import random

from make_data2 import Datasetup

#pkl파일 불러오기

In [ ]:
def iter_json_paths(root: str | Path, pattern: str = "*.pkl"):
    root = Path(root)
    # rglob 결과를 정렬해서 순서를 고정
    for p in sorted(root.glob(pattern)):
        yield p

def load_pkl_file(path: Path):
    try:
        with path.open("rb") as f:
            return pickle.load(f), path

    except pickle.UnpicklingError as e:
        return {"__error__": "PKLDecodeError", "message": str(e)}, path
    except OSError as e:
        return {"__error__": "OSError", "message": str(e)}, path

def iter_pkl_objects(root: str | Path, pattern: str = "*.pkl*"):
    for p in iter_json_paths(root, pattern):
        data, path = load_pkl_file(p)
        yield data, path

In [ ]:
ROOT = Path('/content/drive/MyDrive/law_ai/pickle_final')

train_list = []
test_list = []
val_list = []

folder_to_list = {
    "train": train_list,
    "test": test_list,
    "val": val_list,
}


for folder_entry in sorted(ROOT.iterdir(), key=lambda p: p.name):
    print(folder_entry)
    if not folder_entry.is_dir():
        continue

    folder_name = folder_entry.name

    if folder_name.startswith("train"):
        target_list = train_list
    elif folder_name.startswith("test"):
        target_list = test_list
    elif folder_name.startswith("val"):
        target_list = val_list
    else:
        print("매핑되지 않은 폴더:", folder_name)
        continue

    for obj, p in iter_pkl_objects(folder_entry):
        print(p, len(obj))
        if obj:
            target_list.extend(obj)

In [ ]:
len(train_list)

#test, val 데이터 축소

train 데이터가 6400개는 부족하다 생각하여 test와 val 데이터를 각각 100개만 남기고,

나머지 데이터는 train 데이터에 합치기

In [ ]:
# target에서 무작위로 n개를 추출하고, 추출된 데이터는 target 제거한 뒤 반환
def sample_from_test(target, n: int):
    if n > len(target):
        raise ValueError(f"요청한 샘플 수(n={n})가 target 길이({len(target)})보다 큽니다.")

    # target 인덱스 중 n개 랜덤 선택
    indices = random.sample(range(len(target)), n)

    # 인덱스 정렬 (삭제 시 꼬이지 않도록 뒤에서부터 제거)
    indices.sort(reverse=True)

    sampled_items = []

    for idx in indices:
        sampled_items.append(target[idx])
        del target[idx]   # target 실제 삭제

    return sampled_items


In [ ]:
new_val_list = sample_from_test(val_list, 100)
new_test_list = sample_from_test(test_list, 100)

In [ ]:
print(f'test_list:{len(test_list)}, new_test_list:{len(new_test_list)}')
print(f'val_list:{len(val_list)}, new_val_list:{len(new_val_list)}')

test_list:461, new_test_list:100
val_list:940, new_val_list:100


In [ ]:
len(train_list)

In [ ]:
train_list.extend(test_list)
train_list.extend(val_list)

In [ ]:
len(train_list)

#데이터를 pkl로 저장

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/new_pickle/train.pkl', 'wb') as file:
    pickle.dump(train_list, file)

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/new_pickle/test.pkl', 'wb') as file:
    pickle.dump(new_test_list, file)

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/new_pickle/val.pkl', 'wb') as file:
    pickle.dump(new_val_list, file)

#저장된 파일 로드 후 확인

In [ ]:
with open('/content/drive/MyDrive/law_ai/new_pickle/train.pkl', 'rb') as f:
            data = pickle.load(f)

In [ ]:
len(data)

In [ ]:
with open('/content/drive/MyDrive/law_ai/new_pickle/test.pkl', 'rb') as f:
            data = pickle.load(f)

In [ ]:
len(data)

In [ ]:
with open('/content/drive/MyDrive/law_ai/new_pickle/val.pkl', 'rb') as f:
            data = pickle.load(f)

In [ ]:
len(data)